## *Seminar 2: Theoretical minimum*

## 1. Random processes

#### 1.1 Main definitions

 <u> **Definition 1** </u> A **random process** $X$ is a family of random variables $X_t(\omega)$, $t\in[0,T]$, defined on the same probability space.

We have finite $T$, but in general one can consider processes on $[0,\infty)$.



 <u> **Definition 2** </u> A process $X$ is called **continuous**, if for almost each $\omega$ its trajectory $t\mapsto X_t(\omega)$ is a continuous function.


Similarly we can define processes which are right-continuous, left-continuous, non-decreasing, non-increasing, etc. 


 <u> **Definition 3** </u> Processes $X$ and $Y$ are called **indistinguishable**, if their trajectories coincide a.s., i.e.
$$
P(\omega : X_t(\omega) = Y_t(\omega)\ \text{for all}\ t\ge 0) = 1.
$$

A process $Y$ is called a **modification** of a process $X$, if
$$
X_t = Y_t\ \text{a.s.\ for all}\ t\ge 0.
$$

<u> **Proposition 1** </u>  If $X$ are $Y$ right-continuous (or left-continuous) and $Y$ is a modification of $X$, then $X$ and $Y$ are indistinguishable.

 <u> *Proof idea:* </u>
 
$$ \{ X_t=Y_t\ \text{for all}\ t\ge 0\} = \bigcap_{t\in\mathbb{Q}_+} \{X_t=Y_t\}. $$

<u> **Definition 4** </u> A process $X$ is called **measurable**, if the mapping $X(\omega,t) = X_t(\omega)$ from $\Omega\times\R_+$ to $\mathbb{R}$ is $\mathcal{F} \times B([0,T])$-measurable.


 <u> **Proposition 2** </u> If a process is right- or left-continuous, then it is measurable.


 <u> *Proof idea:* </u> A right-continuous process $X$ can be approximated by piecewise constant processes $X^n_t(\omega) = X(\omega, \lceil nt \rceil/n)$, which are measurable, i.e. for any $t$ and $\omega$

$$
\lim_{n\to\infty} X_t^n(\omega) = X_t(\omega).
$$
Because passing to the limit preserves the measurability, $X$ is measurable.  Similarly for left-continuous processes.


#### 1.2 Classes of measurable processes

With respect to a filtration $\mathcal{F}$, a process $X$ is called

1. **adapted**, if $X_t$ is $\mathcal{F}_t$-measurable for each $t\in[0,T]$;

2. **progressively measurable**, if for any $T'\in[0,T]$ the mapping $X(\omega,t)$, where $t\in[0,T']$, is $\mathcal{F}_{T'}\times B([0,T'])$-measurable;

3. **progressively measurable**, if the mapping $X(\omega,t)$ is measurable with respect to the optional $\sigma$-algebra $\mathcal{O}$, which is generated by all adapted **right**-continuous processes $X$ considered as mappings $X(\omega,t)$;

4. **predictable**, if the mapping $X(\omega,t)$ is measurable with respect to the predictable $\sigma$-algebra $\mathcal{P}$, which is generated by all adapted **left**-continuous processes $X$ considered as mappings $X(\omega,t)$.


 <u> **Proposition 3** </u> The classes of random process satisfy the following inclusions:
$$
\text{predictable} \subseteq \text{optional} \subseteq
\text{prog.\ measurable} \subseteq \text{adapted}.
$$

Any progressively measurable process is measurable and adapted.
But an adapted process can, in general, be non-measurable. Example: $X_t=f(t)$ for some non-measurable non-random function $f$.

#### 1.3 Brownian motion

<u> **Definition 5** </u> A process $X$ is called a **martingale** with respect to a filtration $\mathcal{F}$ if

1. $X$ is adapted to $\mathcal{F}$;
2. $ E|X_t| < \infty$ for any $t\in[0,T]$;
3. $ E(X_t | \mathcal{F}_s) = X_s$ for any $0\le s\le t \le T$.

In a similar way we define **submartingales** ($\geq$) and **supermartingales** ($\leq$).


 <u> **Definition 6** </u> A process $X$ has **independent increments** with respect to a filtration $\mathcal{F}$ if $(X_t-X_s)$ and $\mathcal{F}_s$ are independent for all $0\leq s \leq t\leq T$.


 <u> **Proposition 4** </u> If $X$ is adapted to $\mathcal{F}$, has independent increments and $E X_t = c$ for all $t$, then $X_t$ is a martingale.
 
 <u> *Proof idea:*</u>  $E(X_t - X_s | \mathcal{F}_s) = E(X_t - X_s) = 0$.


 <u> **Definition 7** </u> A **Brownian motion** (or **Wiener process**) with respect to a filtration $\mathcal{F}$ is an adapted process $W$ such that

1. $W_0 = 0$,
2. $W$ has independent increments with respect to $\mathcal{F}$,
3. $W_t - W_s$ has normal distribution with mean 0 and variance $t-s$ for all $s\le t$;
4. $W$ has continuous trajectories with probability 1.

When a filtration in not specified, we mean that $W$ is a Brownian motion with respect to its natural filtration.

From Kolmogorov's continuity theorem, it can be proved that if a process $W$ satisfies properties 1--3, then it has a continuous modification.
Hence, property 4 is sometimes not included in the definition.


**Properties of trajectories of a Brownian motion**

A) It is well-known that with probability 1 trajectories of a Brownian motion

1. are not differentiable at any point $t\ge 0$;
2. do not have points of increase of decrease.
3. have unbounded variation on any interval $[a,b]$, where $b>a$;


B) Let $W$ be a Brownian motion. Then for $0\le t_1<t_2<\dots<t_n$ the random vector $(W_{t_1},\dots,W_{t_n})$ has the Gaussian distribution with 
$$E W_{t_i} = 0, \qquad cov(W_{t_i},W_{t_j}) = \min(t_i,t_j).$$

<u> **Definition 8** </u> The **geometric Brownian motion** with drift $\mu$ and volatility $\sigma$ is the process
$$
S_t = S_0 \exp \{ \sigma W_t + (\mu-\frac{\sigma^2}{2})t \}, \qquad S_0>0.
$$

 <u> **Proposition 5** </u> If $\mu=0$, then $S_t$ is a martingale. If $\mu>0$, then $S_t$ is a submartingale. If $\mu<0$, then $S_t$ is a supermartingale.

## 2. Itô's formula

#### 2.1 Itô's processes

An Ito process is a continuous adapted process which can be represented in the form 

$$X_t = x + \int_{0}^{t} G_s ds + \int_{0}^{t} H_s dW_S, \qquad t \in [ 0, T],$$

where $G, H$ are measurable adapted processes such that 

$$\int_{0}^{T} |G_t| dt < \infty, \qquad \int_{0}^{T} H_t^2 dt < \infty, \qquad a.s.$$

In the differential form, it is writen as 

$$dX_t = G_t dt + H_t dW_t, \qquad X_0 = x.$$

#### 2.2 Itô's lemma

 <u> **Theorem 1** </u>

Let $X_t$ be an Ito process, $f(t, x) \in C^{1, 2}$. Then 


$$
\boxed{
\begin{align*}
df(t, X_t) &= f'_t(t, X_t) dt + f'_x(t, X_t) dX_t + \frac{1}{2} f''_{xx}(t, X_t) (dX_t)^2 \\
&= \left( f'_t(t, X_t) + f'_x(t, X_t) G_t + \frac{1}{2} f''_{xx}(t, X_t) H_t^2 \right) dt + f'_x(t, X_t) H_t dW_t,
\end{align*}
}
$$

where $(dX_t)^2 = H_t^2 dt$.

In integral represenation, the formula is 

$$
\boxed{
\begin{align*}
f(t, X_t) &= f(0, X_0) + \int_{0}^{t} \left(f'_t(t, X_t) + f'_x(t, X_t) G_t + \frac{1}{2} f''_{xx}(t, X_t) H_t^2 \right) dt + \int_{0}^{t} f'_x(t, X_t) H_t dW_t.
\end{align*}
}
$$

#### 1.3 Examples

1. The **geometric Brownian motion** process is defined as

$$
X_t = x_0 \exp \{ (\mu - \sigma^2 / 2) t + \sigma W_t\}, \qquad x_0>0.
$$
Applying Ito's formula to $f(t, W_t)$,
$$
f(t,y) = x_0 \exp \{ (\mu - \sigma^2 / 2) t + \sigma y\}
$$
we see that $X_t$ satisfies the equation
$$
dX_t = \mu X_t dt + \sigma X_t dW_t, \qquad X_0=x_0.
$$

In finance it is used in Black-Scholes model.

2. The **Ornstein-Uhlenbeck** process is defined as

$$
X_t = e^{-\mu t}x_0 + \sigma e^{-\mu t}\int_0^t e^{\mu s} dW_s, \qquad x_0>0.
$$
Applying Ito's formula to $f(t,Y_t)$,
$$
f(t,y) = e^{-\mu t}x_0 + \sigma e^{-\mu t} y, \qquad
dY_t = e^{\mu t} dW_t, \quad Y_0=0,
$$
we see that $X_t$ satisfies the equation
$$
dX_t = -\mu X_t dt + \sigma dW_t, \qquad X_0=x_0.
$$

In finance it can be used, for instance, to model interest rates (the Vasicek model).

## 3. Arbitrage

Assume that on a filtered probaiblity space $(\Omega, \mathcal{F}, (\mathcal{F_t})_{t=0}^{T}, P)$ we observe a market consisting of $1$ riskless asset and $N$ risky assets:

$$dB_t = r_t B_t dt,  \qquad {i.e.} \qquad B_t = B_0 \exp \{ \int_{0}^{t} r_u du\},$$

$$dS_t^n = \mu_t^n dt + \sum_{i=1}^{d} \sigma_t^{ni} dW_t^i,$$

where processes $r_t, \mu_t^n, \sigma^{ni}_t$ are well-defined and risky assets $S_t^n$ continuously pay dividends with rate $q_t^n$. 

<u> **Definition 9** </u> 

A **trading strategy** is a predictable process $\pi_t = (G_t, H_t^1, \dots, H_t^N)$ such that the following integrals are well defined 

$$\int_{0}^{t} G_u dB_u, \qquad \int_{0}^{t} H_u^n dS_u^n, \qquad \int_{0}^{t} H_u^n q_u^n S_u^n du, \qquad t \in [0, T].$$

The **value of the portfolio** is the process 

$$V_t = G_tB_t + \sum_{n=1}^{d} H_t^n S_t^n.$$

A strategy is called **self-financing**, if

$$dV_t = G_t dB_t + \sum_{n=1}^{N} H_t^n dS_t^n + \sum_{n=1}^{N} H_t^n q_t^n S_t^n dt.$$

<u> **Definition 10** </u> 

The **no-arbitrage condition (NA)** means that there is no strategy $\pi_t$ such that 

1. $V_0^{\pi} = 0$ a.s.,
2. $V_T^{\pi} \geq 0$ a.s.,
3. $P(V_T^{\pi} > 0) > 0$ a.s. 

<u> **Definition 11** </u> 

An **equivalent martingale measure (EMM)** is a probability measure $\tilde P ∼ P $ such that the discounted dividend-adjusted prices

$$S_t^{*, n} = \frac{S_t^n e^{\int_{0}^{t} q_u^n du }}{B_t / B_0} = S_t^n e^{\int_{0}^{t} (q_u^n - r_u) du}$$ 

are martingales with repsect to $\tilde P$ for each $n = 1, \dots, N$.

In discrete time,

$$NA \iff \exists \text{ equivalent martingale measure } \tilde P $$

What happens in continuous time setup? 

<u> **Example** </u> 

Consider the BS model with 
$$S_t = \exp \{ \mu t + \sigma W_t\}, \qquad B_t =  e^{rt}$$

Let $r = 0$ and $\mu = - \sigma^2 / 2$. Then $S_t$ is a martingale and 
$$dS_t = S_t \sigma dW_t,$$
which leads to 
$$S_t = 1 + \sigma \int_{0}^{t} S_r dW_r.$$


It is well known that $W_t$ almost surely reachs any fixed level for finite time. That is, $\tau := \inf \{ t \geq 0: W_t = 1\}$ is finite a.s. 

Let $\xi_t = \dfrac{1}{1 - t},\quad t < 1$. 

Then if $X_t = \int_{0}^{t} \xi_r dW_r$, we get that $Law(X_t) = Law(W_{\frac{t}{1 - t}})$.

Hence, $X_t$ almost surely on $[0, 1)$ crosses the level $1$, that is $\tau := \inf \{ t \in [0, 1): X_t = 1\} < \infty$ a.s.

Let $\tilde \xi = \xi  \mathbb{I}_{ [0, \tau ]}$ and consider a self-financing strategy $\pi_t = (G_t, H_t^{1})$, such that $V_{0}^{\pi} = 0$ and $H_t^1 := \frac{\tilde \xi_t}{S_t}$. 

Then 
$$V_1^{\pi} = V_0^{\pi} + \int_{0}^{1} H_t^1 dS_t + \int_{0}^{1} G_t dB_t = \int_{0}^{1} G_t dB_t = \int_{0}^{1} \frac{\tilde \xi_t}{S_t} dS_t = \int_{0}^{1} \frac{\tilde \xi_t}{S_t} \sigma S_t dW_t = \sigma \int_{0}^{1} \tilde \xi_t dW_t = 1.$$

Hence, we get a strategy which transfers $0 \$$ to $1 \$$:

$$\pi: V_0^{\pi} = 0 \mapsto V_1^{\pi} = 1.$$

and we get an aribtrage opportunity. But this is not the end. Consider now the strategy $\tilde \pi = (1 - G_t, - H_t^{1})$. Then $V_0^{\tilde \pi} = 1$ and $\tilde V_1^{\pi} = 0$. So we have a 'suicide' strategy which transfers $1 \$$ to $0 \$$:

$$\tilde \pi: V_0^{\pi} = 1 \mapsto V_1^{\pi} = 0.$$

For that reason we need an extra condition.


<u> **Definition 12** </u> 

The strategy is **admissible**, if $\exists c \in \mathbb{R}$ such that $\forall t \geq 0$ we get that $V_t^{\pi} \geq c.$

Why in example above this condition is violated? Exactly because the process $X_t$ for finite time can take arbitrarily large values, as $Law(X_t) = Law(W_{t/(1-t)})$. 

<u> **Definition 13** </u> 

The **no-free-lunch-with-vanishing-risk condition (NFLVR)** means that there is no sequencey $X^n \in L^{\infty} (\mathcal{F}_T)$ such that 

1. $X^n \leq V^{\pi^n}_T$ for some $\pi^n$, $V_0^{\pi^n} = 0$;
2. $X^n \to X$ in $L^{\infty} (\mathcal{F}_t)$;
3. $X \geq 0$;
4. $P(X > 0) > 0$.

"Vanishing risk" means that $\| X^n - X \|_{L^{\infty}} \to 0$, that is for some $\varepsilon > 0$ $\exists N$: $\forall n > N$ we have $|X^n - X| < \varepsilon$ a.s., which means that the maximu value which can be lost is $\varepsilon$ which tends to $0$ when $n \to \infty$.

<u> **Definition 14** </u> 

An equivalent local martingale measure (ELMM) is a probability measure $\tilde P \sim P$ such that the discounted dividend-adjusted prices
$S^{∗,n}$ re local martingales with respect to $\tilde P$.

<u> **Proposition 6**</u> 

An ELMM is a probability measure $\tilde P \sim P$ such that for any strategy $π_t$ the discounted value $V_t^{*, \pi} = \dfrac{V_t^{\pi}}{B_t}$
are local martingales with respect to $\tilde P$.

<u> *Proof idea:*</u> 

Let $q_t \equiv 0$. Then

$$d V_t^{*, \pi} = V_t^{\pi} dB_t^{-1} + B_t^{-1} dV_t^{\pi} = V_t^{\pi} dB_t^{-1} + B_t^{-1} G_t dB_t + B_t^{-1} \sum_{n=1}^{N} H_t^n dS_t^n.$$

$$\sum_{n=1}^{N} H_t^n dS_t^{*, n} = \sum_{n=1}^{N} H_t^n \left( S_t^{n} dB^{-1}_t + B^{-1}_t dS_t^n\right) = V_t^{\pi} dB^{-1}_t - G_t B_t dB_t^{-1} + \sum_{n=1}^{N} H_t^n  B^{-1}_t dS_t^n.$$

Hence, $$d V_t^{*, \pi} = \sum_{n=1}^{N} H_t^n dS_t^{*, n}.$$

So if $S_t^{*, n}$ is local martingale, $V_t^{*, \pi}$ is also a local martignale.

In continuous time, 

$$\text{No Arbitrage} \mapsto \text{No Free Lunch With Vanishing Risk}$$

$$\text{Martingality} \mapsto \text{Local Martingality}$$

**Intuition:** If $V_t^{\pi} / B_t$ is local martingale and $V_t^{\pi} \geq c$, then $V_t^{\pi} / B_t$ is supermartingale, which means that 

$$E^{\tilde P} \left[  V_t^{\pi} / B_t | \mathcal{F_s} \right] \leq V_s^{\pi} / B_s \qquad s \leq t,$$

and so "on average" $V_t^{\pi} / B_t$ is not greater than $V_s^{\pi} / B_s$, wcich is "no arbitrage".